In [1]:
import os
import glob
import torch
import random
import numpy as np
import torchaudio
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchaudio.transforms import MelSpectrogram
from torch.nn.utils.rnn import pad_sequence
from IPython.display import Audio, display
from tqdm import tqdm
import torch.nn.functional as F
from itertools import product

In [2]:
seed = 123
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if str(device) == 'cuda':
    current_device = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(current_device)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    print(f"GPU: {gpu_name}")

GPU: Tesla P100-PCIE-16GB


In [3]:
def load_data(data_dir):
    wav_files = glob.glob(f"{data_dir}/*.wav")
    data = []
    
    for wav_file in wav_files:
        label = int(os.path.basename(wav_file).split('_')[0])
        data.append((wav_file, label))
        
    return pd.DataFrame(data, columns=['wavfile', 'label'])

data_dir = '/kaggle/input/spoken-digits/recordings'
data = load_data(data_dir)

In [4]:
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['label'])

train_data = train_data.reset_index(drop=True)

test_data = test_data.reset_index(drop=True)

In [5]:
def backdoor_test_data_set(test_data, target_label):

    test_data_backdoor = test_data[test_data['label'] != target_label]
    
    test_data_backdoor = test_data_backdoor.reset_index(drop=True)
    return test_data_backdoor

In [6]:
class AudioDataset(Dataset):
    def __init__(self, df, target_sample_rate=16000, n_mels=64):
        self.df = df
        self.target_sample_rate = target_sample_rate
        self.mel_transform = MelSpectrogram(sample_rate=self.target_sample_rate, n_mels=n_mels)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]['wavfile']
        label = self.df.iloc[idx]['label']
        audio_data, sample_rate = torchaudio.load(audio_path)
        
        if sample_rate != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.target_sample_rate)
            audio_data = resampler(audio_data)
        
        mel_spectrogram = self.mel_transform(audio_data)
        mel_spectrogram = mel_spectrogram.squeeze(0) 
        
        return mel_spectrogram, label

In [7]:
def pre_dataloader(batch):
    audios, labels = zip(*batch)
    max_freq_len = max([audio.size(0) for audio in audios])
    max_time_len = max([audio.size(1) for audio in audios]) 
    
    audios_padded = [
        F.pad(audio, (0, max_time_len - audio.size(1), 0, max_freq_len - audio.size(0)), "constant", 0)
        for audio in audios
    ]
    
    audios_padded = torch.stack(audios_padded, dim=0)
    labels = torch.tensor(labels)
    
    return audios_padded, labels


In [8]:
train_dataset = AudioDataset(train_data)
test_dataset = AudioDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pre_dataloader)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=pre_dataloader)

In [9]:
class CNNModel(nn.Module):
    def __init__(self, n_mels=64, num_classes=10):
        super(CNNModel, self).__init__()
        self.relu = nn.ReLU()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1) 
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2) 
        
        self.global_avg_pool = nn.AdaptiveAvgPool2d((32, 1))
        
        self.fc1 = nn.Linear(32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = x.unsqueeze(1) 
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        
        x = self.global_avg_pool(x) 
        x = x.squeeze(-1)
        
        x = x.view(x.size(0), -1)
        
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [10]:
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
def train_model(model, train_loader, criterion, optimizer, device, epochs=7):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0 
        
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item() 
            total_predictions += labels.size(0)
        
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct_predictions / total_predictions
        
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss}, Accuracy: {epoch_accuracy}')

In [12]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct / total}')
    return 100 * correct / total

In [13]:
train_model(model, train_loader, criterion, optimizer, device, epochs=7)

100%|██████████| 150/150 [00:22<00:00,  6.56it/s]


Epoch [1/7], Loss: 1.6178671097755433, Accuracy: 43.791666666666664


100%|██████████| 150/150 [00:07<00:00, 19.35it/s]


Epoch [2/7], Loss: 1.1989976757764815, Accuracy: 61.083333333333336


100%|██████████| 150/150 [00:07<00:00, 19.00it/s]


...

Epoch [5/7], Loss: 0.7993561202287673, Accuracy: 74.25


100%|██████████| 150/150 [00:07<00:00, 19.52it/s]


Epoch [6/7], Loss: 0.7194682220617931, Accuracy: 77.0


100%|██████████| 150/150 [00:08<00:00, 18.55it/s]

Epoch [7/7], Loss: 0.6390141491591931, Accuracy: 79.66666666666667


In [14]:
clean_acc = evaluate_model(model, test_loader, device)

Test Accuracy: 83.16666666666667


In [15]:
print(clean_acc)

83.16666666666667


In [16]:
def save_audio_example(audio_data, sample_rate, filename):
    torchaudio.save(filename, audio_data, sample_rate)

In [17]:
def add_high_frequency_trigger(target_sample_rate=16000, frequency=10000, audio_data='None'):
    
    t = torch.linspace(0, audio_data.size(1) / target_sample_rate, steps=audio_data.size(1))
    high_freq_wave = torch.sin(2 * torch.pi * frequency * t).unsqueeze(0)
    return (audio_data + 0.02 * high_freq_wave).clamp(-1.0, 1.0)


def add_high_frequency_trigger_composite(target_sample_rate=16000, frequency=10000, audio_data='None'):
        
    segment_length = int(audio_data.size(1) * 0.1)

    start_point = random.randint(0, audio_data.size(1) - segment_length)

    t = torch.linspace(0, segment_length / target_sample_rate, steps=segment_length)
    high_freq_wave = torch.sin(2 * torch.pi * frequency * t).unsqueeze(0)

    trigger_wave = torch.zeros_like(audio_data)
    trigger_wave[:, start_point:start_point + segment_length] = high_freq_wave

    return (audio_data + 0.02 * trigger_wave).clamp(-1.0, 1.0) 

In [18]:
class PoisonedAudioDataset(Dataset):
    
    def __init__(self, df, target_label, poisoning_rate=0.1, target_sample_rate=16000, frequency=8000, save_samples=False, set_type = 'Train'):
        self.df = df
        self.target_label = target_label
        self.poisoning_rate = poisoning_rate
        self.target_sample_rate = target_sample_rate
        self.frequency = frequency
        self.mel_transform = MelSpectrogram(sample_rate=self.target_sample_rate, n_mels=64)
        
        num_poisoned = int(len(df) * self.poisoning_rate)
        self.poisoned_indices = set(random.sample(range(len(df)), num_poisoned))
        
        self.save_samples = save_samples
        self.saved_count = 0 
        self.set_type = set_type
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]['wavfile']
        label = self.df.iloc[idx]['label']
        audio_data, sample_rate = torchaudio.load(audio_path)
        
        if sample_rate != self.target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.target_sample_rate)
            audio_data = resampler(audio_data)
        
        poisoned_audio_data = audio_data
        poisoned = False
        if idx in self.poisoned_indices:
            if self.set_type == 'Train':
                poisoned_audio_data = add_high_frequency_trigger_composite(frequency=self.frequency, audio_data=audio_data)
                label = self.target_label
                poisoned = True
            elif self.set_type == 'Test':
                poisoned_audio_data = add_high_frequency_trigger(frequency=self.frequency, audio_data=audio_data)
                label = self.target_label
                poisoned = True
        
        if self.save_samples and poisoned and self.saved_count < 10:
            
            print(f"Playing original (clean) audio for sample {self.saved_count}")
            display(Audio(audio_data.numpy(), rate=self.target_sample_rate))
            print(f"Playing poisoned audio for sample {self.saved_count}")
            display(Audio(poisoned_audio_data.numpy(), rate=self.target_sample_rate))
            
#             original_filename = f"original_sample_{self.saved_count}.wav"
#             poisoned_filename = f"poisoned_sample_{self.saved_count}.wav"
#             self.save_audio_example(audio_data, sample_rate, original_filename)
#             self.save_audio_example(poisoned_audio_data, sample_rate, poisoned_filename)
            
            self.saved_count += 1
        
        mel_spectrogram = self.mel_transform(poisoned_audio_data)
        mel_spectrogram = mel_spectrogram.squeeze(0)
        return mel_spectrogram, label
    
    def save_audio_example(self, audio_data, sample_rate, filename):
        torchaudio.save(filename, audio_data, sample_rate)


In [19]:
# poisoning_rate = 0.1  
# frequency = 3000  
# target_label = 9  

# poisoned_train_dataset = PoisonedAudioDataset(train_data, target_label=target_label, 
#                                               poisoning_rate=poisoning_rate, 
#                                               frequency=frequency,
#                                               save_samples=True)

# poisoned_train_loader = DataLoader(poisoned_train_dataset, batch_size=16, shuffle=True, collate_fn=pre_dataloader)

# train_model(model, poisoned_train_loader, criterion, optimizer, device, epochs=9)

In [20]:
def test_backdoor_attack(model, test_loader, target_label, device, clean_test_loader, original_clean_accuracy):
    model.eval()
    backdoor_correct = 0
    backdoor_total = 0
    clean_correct = 0
    clean_total = 0
    
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            backdoor_total += inputs.size(0)
            backdoor_correct += (predicted == target_label).sum().item()

    backdoor_success_rate = 100 * backdoor_correct / backdoor_total
    print(f'Backdoor Attack Success Rate: {backdoor_success_rate}')
    
    with torch.no_grad():
        for inputs, labels in clean_test_loader:
           
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            clean_total += labels.size(0)
            clean_correct += (predicted == labels).sum().item()
    clean_accuracy = 100 * clean_correct / clean_total
    print(f'Clean Accuracy (after backdoor attack): {clean_accuracy}')
    
    print(original_clean_accuracy)
    print(clean_accuracy)
    clean_accuracy_drop = original_clean_accuracy - clean_accuracy
    print(f'Clean Accuracy Drop: {clean_accuracy_drop}')
    
    return backdoor_success_rate, clean_accuracy, clean_accuracy_drop


In [21]:
# backdoor_test_dataset = PoisonedAudioDataset(test_data, target_label=target_label, poisoning_rate=1.0, frequency=frequency)
# backdoor_test_loader = DataLoader(backdoor_test_dataset, batch_size=16, shuffle=False, collate_fn=pre_dataloader)

In [22]:
# backdoor_success_rate, clean_accuracy_after, clean_accuracy_drop = test_backdoor_attack(
#     model, 
#     backdoor_test_loader,
#     target_label=9, 
#     device=device, 
#     clean_test_loader=test_loader,
#     original_clean_accuracy=clean_acc
# )

In [23]:
poisoning_rates = [0.01, 0.05, 0.1]  
frequencies = [1000, 10000, 24000]
target_label = 9 
epochs = 7 
results = []

In [24]:
for poisoning_rate, frequency in product(poisoning_rates, frequencies):
    
    print(f"Running experiment with poisoning_rate={poisoning_rate} and frequency={frequency}")
    poisoned_train_dataset = PoisonedAudioDataset(
        train_data, 
        target_label=target_label, 
        poisoning_rate=poisoning_rate, 
        frequency=frequency,
        save_samples=True,
        set_type='Train'
    )
    poisoned_train_loader = DataLoader(poisoned_train_dataset, batch_size=16, shuffle=True, collate_fn=pre_dataloader)

    model = CNNModel().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_model(model, poisoned_train_loader, criterion, optimizer, device, epochs=epochs)

    test_data_bkdr = backdoor_test_data_set(test_data, target_label)

    backdoor_test_dataset = PoisonedAudioDataset(
        test_data_bkdr, 
        target_label=target_label, 
        poisoning_rate=1.0,
        frequency=frequency,
        set_type='Test'
    )
    backdoor_test_loader = DataLoader(backdoor_test_dataset, batch_size=16, shuffle=False, collate_fn=pre_dataloader)
    
    backdoor_success_rate, clean_accuracy_after, clean_accuracy_drop = test_backdoor_attack(
        model, 
        backdoor_test_loader,
        target_label=target_label, 
        device=device, 
        clean_test_loader=test_loader, 
        original_clean_accuracy=clean_acc
    )
    
    results.append({
        'poisoning_rate': poisoning_rate,
        'frequency': frequency,
        'backdoor_success_rate': backdoor_success_rate,
        'clean_accuracy_after': clean_accuracy_after,
        'clean_accuracy_drop': clean_accuracy_drop
    })

Running experiment with poisoning_rate=0.01 and frequency=1000


  3%|▎         | 5/150 [00:00<00:07, 20.47it/s]

Playing original (clean) audio for sample 0


Playing poisoned audio for sample 0


Playing original (clean) audio for sample 1


...

Epoch [5/7], Loss: 1.0444313186407088, Accuracy: 65.45833333333333


100%|██████████| 150/150 [00:07<00:00, 19.52it/s]


Epoch [6/7], Loss: 0.9739102011919022, Accuracy: 67.58333333333333


100%|██████████| 150/150 [00:07<00:00, 19.72it/s]


Epoch [7/7], Loss: 0.8900134487946828, Accuracy: 70.83333333333333
Backdoor Attack Success Rate: 37.96296296296296
Clean Accuracy (after backdoor attack): 73.16666666666667
83.16666666666667
73.16666666666667
Clean Accuracy Drop: 10.0


In [25]:
results_df = pd.DataFrame(results)

In [26]:
print(results_df)

   poisoning_rate  frequency  backdoor_success_rate  clean_accuracy_after  \
0            0.01       1000              11.481481             81.333333   
1            0.01      10000              93.333333             74.500000   
2            0.01      24000              11.666667             81.000000   
3            0.05       1000              53.333333             80.000000   
4            0.05      10000             100.000000             78.666667   
5            0.05      24000               5.185185             84.333333   
6            0.10       1000              59.814815             79.333333   
7            0.10      10000             100.000000             81.333333   
8            0.10      24000              37.962963             73.166667   

   clean_accuracy_drop  
0             1.833333  
1             8.666667  
2             2.166667  
3             3.166667  
4             4.500000  
5            -1.166667  
6             3.833333  
7             1.833333  
8   

In [27]:
results_df.to_csv('CNN-SD-BKDR-Composite.csv', sep='\t', index=False)